### Quick start to Tensorflow 2 - Basic MNIST classifier using CNN
Original [doc](https://www.tensorflow.org/tutorials/quickstart/advanced)

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
tf.__version__

'2.2.0'

___
#### Checking available datasets with keras

In [27]:
#Checking available datasets with keras
for dataset in dir(tf.keras.datasets):
    if "_" not in dataset:
        print(dataset)

cifar10
cifar100
imdb
mnist
reuters


---
#### load and prepare MNIST dataset

In [28]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0
x_train = x_train[..., tf.newaxis].astype("float32")
#OR x_train = x_train[..., np.newaxis].astype("float32")
#OR x_train = x_train.reshape(x_train.shape[0], 28, 28, -1).astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

**Note:** *...* is called ellipsis and is used for slicing entire array and is equivalent to *:,:,:* in this case

Refer second answer [here](https://stackoverflow.com/questions/118370/how-do-you-use-the-ellipsis-slicing-syntax-in-python) for more details
___